In [37]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this note

In [38]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
        # set number of nodes in each input, hidden, output layer
        self.inputNodes = inputNodes
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.hiddenNodes, -0.5), (self.hiddenNodes, self.inputNodes))
        self.who = numpy.random.normal(0.0, pow(self.outputNodes, -0.5), (self.outputNodes, self.hiddenNodes))

        # learning rate
        self.lr = learningRate
        
        # activation function is the sigmoid function
        self.activationFunction = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputsList, targetsList):
        # convert inputs list to 2d array
        inputs = numpy.array(inputsList, ndmin=2).T
        targets = numpy.array(targetsList, ndmin=2).T
        
        # calculate signals into hidden layer
        hiddenInputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hiddenOutputs = self.activationFunction(hiddenInputs)
        
        # calculate signals into final output layer
        finalInputs = numpy.dot(self.who, hiddenOutputs)
        # calculate the signals emerging from final output layer
        finalOutputs = self.activationFunction(finalInputs)
        
        # output layer error is the (target - actual)
        outputErrors = targets - finalOutputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hiddenErrors = numpy.dot(self.who.T, outputErrors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((outputErrors * finalOutputs * (1.0 - finalOutputs)), numpy.transpose(hiddenOutputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hiddenErrors * hiddenOutputs * (1.0 - hiddenOutputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputsList):
        # convert inputs list to 2d array
        inputs = numpy.array(inputsList, ndmin=2).T
        
        # calculate signals into hidden layer
        hiddenInputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hiddenOutputs = self.activationFunction(hiddenInputs)
        
        # calculate signals into final output layer
        finalInputs = numpy.dot(self.who, hiddenOutputs)
        # calculate the signals emerging from final output layer
        finalOutputs = self.activationFunction(finalInputs)
        
        return finalOutputs

In [39]:
# number of input, hidden and output nodes
inputNodes = 784
hiddenNodes = 200
outputNodes = 10

# learning rate
learningRate = 0.1

# create instance of neural network
nueral = neuralNetwork(inputNodes,hiddenNodes,outputNodes, learningRate)

In [40]:
# load the mnist training data CSV file into a list
trainingDataFile = open("/insert path to /training/mnist_train_100.csv", 'r') #here
trainingDataList = trainingDataFile.readlines()
#print (training_data_list)
trainingDataFile.close()

In [41]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in trainingDataList:
        # split the record by the ',' commas
        allValues = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(allValues[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(outputNodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(allValues[0])] = 0.99
        nueral.train(inputs, targets)
        pass
    pass

In [42]:
# load the mnist test data CSV file into a list
testDataFile = open("/insert path to /test/mnist_test.csv", 'r') #here
testDataList = testDataFile.readlines()
#print (testDataList)
testDataFile.close()

In [43]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in testDataList:
    # split the record by the ',' commas
    allValues = record.split(',')
    # correct answer is first value
    correctLabel = int(allValues[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(allValues[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = nueral.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correctLabel):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [44]:
# calculate the performance score, the fraction of correct answers
scorecardArray = numpy.asarray(scorecard)
print ("performance = ", scorecardArray.sum() / scorecardArray.size)

performance =  0.667
